# 📏 Regularização

Regularização é uma técnica fundamental em aprendizado de máquina que visa prevenir o **overfitting**.  

Ela funciona **adicionando um termo de penalização à função de custo do modelo**, que é uma combinação do erro de treinamento e de uma penalidade proporcional à magnitude dos coeficientes do modelo.

---

Essas penalidades:

- ✅ **Incentivam o modelo a manter os coeficientes pequenos**
- ⚖️ **Reduzem a complexidade do modelo**
- 📈 **Aumentam sua capacidade de generalização para novos dados**

---

Além de prevenir o overfitting, a regularização também pode:

- 🧹 **Ajudar a identificar e remover características irrelevantes ou redundantes do modelo**,  
  atribuindo pesos baixos a essas características.

---

### L1 (Lasso – *Least Absolute Shrinkage and Selection Operator*):
Uma penalidade é adicionada à função de custo igual à **soma dos valores absolutos dos coeficientes do modelo**.  
Isso incentiva alguns dos coeficientes a se tornarem exatamente zero, resultando em:

- ✂️ **Seleção automática de características**

---

### L2 (Ridge):
Uma penalidade é adicionada à função de custo igual à **soma dos quadrados dos coeficientes do modelo**.  
Isso:

- ❌ **Desencoraja coeficientes muito grandes**
- 🎯 **Suaviza o ajuste do modelo**
- 📉 **Reduz a variância e melhora a capacidade de generalização**

### 🔷 Elastic Net:
O **Elastic Net** combina os efeitos da regularização **L1 e L2**, adicionando uma **combinação das penalidades L1 e L2** à função de custo.  
Isso permite que o modelo:

- Selecione **características relevantes** (como L1)
- Regularize **coeficientes** (como L2)

👉 Proporcionando um equilíbrio entre **dispersão e suavização**.

---

### 🟢 Lasso Least Angle Regression (LARS):
O **LARS** é um método de regularização que estende o Lasso para permitir a inclusão de **mais variáveis do que observações** no conjunto de dados.  
Ele também é eficiente em:

- ✅ **Lidar com multicolinearidade entre as variáveis**

---

### 🧩 Group Lasso

**Group Lasso** é uma extensão da regularização Lasso que **penaliza grupos inteiros de características**, em vez de características individuais.  

🔹 Isso é útil quando há **grupos de características que estão relacionadas** e devem ser tratadas juntas.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np


### Carregar e visualizar os dados - Regressão

In [2]:
# carregar os dados - regressao
df = pd.read_csv('./datasets/dataset_colesterol.csv')

In [3]:
# visualizar estrutura dos dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  1000 non-null   int64  
 1   Grupo Sanguíneo     996 non-null    object 
 2   Fumante             997 non-null    object 
 3   Nível de Atividade  996 non-null    object 
 4   Idade               997 non-null    float64
 5   Peso                997 non-null    float64
 6   Altura              997 non-null    float64
 7   Colesterol          1000 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 62.6+ KB


In [4]:
# visualizar dataframe
df.head(10)

,Id,Grupo Sanguíneo,Fumante,Nível de Atividade,Idade,Peso,Altura,Colesterol
0,1,B,Sim,Baixo,33.0,85.1,186.0,199.63
1,2,A,Não,Moderado,68.0,105.0,184.0,236.98
2,3,O,Não,Alto,25.0,64.8,180.0,161.79
3,4,A,Não,Alto,43.0,120.2,167.0,336.24
4,5,AB,Não,Baixo,79.0,88.5,175.0,226.23
5,6,B,Não,Baixo,68.0,66.8,170.0,185.31
6,7,A,Sim,Baixo,60.0,117.3,181.0,289.33
7,8,O,Sim,Moderado,35.0,86.9,174.0,216.48
8,9,O,Não,Baixo,62.0,81.3,166.0,235.30
9,10,B,Sim,Alto,44.0,32.7,165.0,97.79


In [5]:
# ajustar dataframe
df.drop('Id', axis=1, inplace=True)

# aplicar onehotencoding nas variaveis categoricas
df = pd.get_dummies(df, columns=['Grupo Sanguíneo', 'Fumante', 'Nível de Atividade'])

In [6]:
df = df.dropna()

In [7]:
# dataframe atualizado
df

,Idade,Peso,Altura,Colesterol,Grupo Sanguíneo_A,Grupo Sanguíneo_AB,Grupo Sanguíneo_B,Grupo Sanguíneo_O,Fumante_Não,Fumante_Sim,Nível de Atividade_Alto,Nível de Atividade_Baixo,Nível de Atividade_Moderado
0,33.0,85.1,186.0,199.63,False,False,True,False,False,True,False,True,False
1,68.0,105.0,184.0,236.98,True,False,False,False,True,False,False,False,True
2,25.0,64.8,180.0,161.79,False,False,False,True,True,False,True,False,False
3,43.0,120.2,167.0,336.24,True,False,False,False,True,False,True,False,False
4,79.0,88.5,175.0,226.23,False,True,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,31.0,68.1,166.0,206.81,False,True,False,False,False,True,False,False,True
996,51.0,47.7,170.0,128.03,False,False,False,True,True,False,True,False,False
997,39.0,85.5,176.0,211.14,False,True,False,False,True,False,False,True,False
998,61.0,91.2,161.0,284.53,False,True,False,False,False,True,False,True,False


### Treinar Modelo de Regressão com Lasso (L1)

In [8]:
# Separar X e y
X = df.drop('Colesterol', axis=1)
y = df['Colesterol']

In [9]:
# Separar Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [13]:
# treinar o modelo de regressao linear multipla com lasso
# quanto maior o alpha, maior a penalizacao e mais coeficientes tendem a ser reduzidos a 0
model_lasso = Lasso(alpha=0.1)
model_lasso.fit(X_train, y_train)

Lasso(alpha=0.1)

In [15]:
# Mostrar Importância de Features
def importancia_features(modelo):
    importance = np.abs(modelo.coef_)
    print('Importância das Features')
    for i, feature in enumerate(modelo.feature_names_in_):
        print(f"{feature}: {importance[i]}")


In [16]:
# Importância das Features – Lasso
importancia_features(model_lasso)

Importância das Features
Idade: 0.011276332707821515
Peso: 2.49177516579548
Altura: 2.179000389781833
Grupo Sanguíneo_A: 0.053110581724528795
Grupo Sanguíneo_AB: 1.4158548631246126
Grupo Sanguíneo_B: 0.0
Grupo Sanguíneo_O: 0.0
Fumante_Não: 0.0
Fumante_Sim: 2.108063661753771
Nível de Atividade_Alto: 0.1817840694875455
Nível de Atividade_Baixo: 0.06227565107522554
Nível de Atividade_Moderado: 0.0


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Função para avaliar performance sem usar squared=False
def performance_regressao(modelo, X_test, y_test):
    y_pred = modelo.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    return rmse

In [19]:
# Performance Regressão com Lasso
performance_regressao(model_lasso, X_test, y_test)

np.float64(8.839489211575424)

In [20]:
# Treinar com LassoCV
model_lasso_cv = LassoCV(alphas=[0.1, 0.5, 1], cv=5, random_state=51)
model_lasso_cv.fit(X_train, y_train)


LassoCV(alphas=[0.1, 0.5, 1], cv=5, random_state=51)

In [ ]:
# Importância das Features – LassoCV
importancia_features(model_lasso_cv)

Importância das Features
Idade: 0.011276332707821515
Peso: 2.49177516579548
Altura: 2.179000389781833
Grupo Sanguíneo_A: 0.053110581724528795
Grupo Sanguíneo_AB: 1.4158548631246126
Grupo Sanguíneo_B: 0.0
Grupo Sanguíneo_O: 0.0
Fumante_Não: 0.0
Fumante_Sim: 2.108063661753771
Nível de Atividade_Alto: 0.1817840694875455
Nível de Atividade_Baixo: 0.06227565107522554
Nível de Atividade_Moderado: 0.0


In [23]:
# Performance Regressão com LassoCV
performance_regressao(model_lasso_cv, X_test, y_test)

np.float64(8.839489211575424)

### Treinar Modelo de Regressão com Ridge (L2)

In [24]:
# Treinar o modelo de regressão linear múltipla com Ridge
# Quanto maior o alpha, maior a penalização e mais coeficientes tendem a ser reduzidos
model_ridge = Ridge(alpha=0.1)
model_ridge.fit(X_train, y_train)

Ridge(alpha=0.1)

In [25]:
# Importância das Features – Ridge
importancia_features(model_ridge)

Importância das Features
Idade: 0.011190460333139403
Peso: 2.484587943547598
Altura: 2.1744854254644577
Grupo Sanguíneo_A: 1.7442353228923175
Grupo Sanguíneo_AB: 0.3715469274873889
Grupo Sanguíneo_B: 1.3767797387412686
Grupo Sanguíneo_O: 1.397345843917695
Fumante_Não: 5.773398636113596
Fumante_Sim: 8.310481655245248
Nível de Atividade_Alto: 2.0936118056802435
Nível de Atividade_Baixo: 2.99789469160786
Nível de Atividade_Moderado: 2.751260878714591


In [27]:
# Performance Regressão com Ridge
performance_regressao(model_ridge, X_test, y_test)

np.float64(8.834914520080362)

In [28]:
# Treinar com RidgeCV
model_ridge_cv = RidgeCV(alphas=[0.1, 0.5, 1], cv=5)
model_ridge_cv.fit(X, y)

RidgeCV(alphas=[0.1, 0.5, 1], cv=5)

In [29]:
# Importância das Features – RidgeCV
importancia_features(model_ridge_cv)

Importância das Features
Idade: 0.01707578177175205
Peso: 2.4744083912489803
Altura: 2.191309166368036
Grupo Sanguíneo_A: 0.38010601492824025
Grupo Sanguíneo_AB: 1.797692537519227
Grupo Sanguíneo_B: 0.2292482188412377
Grupo Sanguíneo_O: 0.47012097594430097
Fumante_Não: 3.2440177018987164
Fumante_Sim: 5.541249191878231
Nível de Atividade_Alto: 1.824136930845972
Nível de Atividade_Baixo: 0.32593631800427153
Nível de Atividade_Moderado: 0.8644194830221513


In [30]:
# Performance Regressão com RidgeCV
performance_regressao(model_ridge_cv, X_test, y_test)

np.float64(8.744283757668793)